In [ ]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib as mplib
import matplotlib.pyplot as plt
import matplotlib.image as img
import itertools
from keras import layers, models
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
from collections import Counter
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('./hmnist_28_28_RGB.csv')
print(data.shape)

In [ ]:
Y = data['label']
print(type(Y))
print(Counter(Y))
nlabels = Y.value_counts().size
print(nlabels)

In [ ]:
X = data.drop(['label'],axis=1)
X = X/255

In [ ]:
print(X.shape)
print(Y.shape)